<a href="https://colab.research.google.com/github/ShokhinaElizaveta/stock-markets-analytics/blob/main/HA_2_for_Module2_Colab_Working_with_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


In [3]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [5]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [6]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      67 non-null     object
 1   Symbol        67 non-null     object
 2   Company Name  67 non-null     object
 3   IPO Price     67 non-null     object
 4   Current       67 non-null     object
 5   Return        67 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [7]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [8]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      221 non-null    object
 1   Symbol        221 non-null    object
 2   Company Name  221 non-null    object
 3   IPO Price     221 non-null    object
 4   Current       221 non-null    object
 5   Return        221 non-null    object
dtypes: object(6)
memory usage: 10.5+ KB


In [9]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [10]:
# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [11]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [12]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [13]:
# Correctly applied transformations with 'defensive' techniques, but now not all are non-null
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      221 non-null    datetime64[ns]
 1   Symbol        221 non-null    object        
 2   Company Name  221 non-null    object        
 3   IPO Price     221 non-null    float64       
 4   Current       221 non-null    float64       
 5   Return        219 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.5+ KB


In [14]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          2
dtype: int64

In [15]:
# Do you want to leave the record or not?
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
7,2024-04-25,MRX,Marex Group plc,19.00,19.19,NaN
199,2023-02-15,GXAI,Gaxos.ai Inc.,4.15,4.10,NaN


In [16]:
# Descriptive Analytics of a dataset
stacked_ipos_df.describe()

,IPO Date,IPO Price,Current,Return
count,221,221.000000,221.000000,219.000000
mean,2023-09-11 11:04:36.923076864,11.082036,11.265566,-0.191961
min,2023-01-13 00:00:00,2.500000,0.000000,-0.999600
25%,2023-04-20 00:00:00,4.000000,1.260000,-0.724200
50%,2023-09-15 00:00:00,8.000000,5.720000,-0.207500
75%,2024-01-26 00:00:00,15.000000,10.930000,0.063650
max,2024-05-09 00:00:00,92.000000,122.660000,2.672500
std,NaN,11.194790,17.077804,0.655152


# Question 1

In [17]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [18]:
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     324 non-null    object
 1   Symbol          324 non-null    object
 2   Company Name    324 non-null    object
 3   Price Range     324 non-null    object
 4   Shares Offered  324 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [19]:
# convert to datetime
ipos_filings['Filing Date'] = pd.to_datetime(ipos_filings['Filing Date'], format='%b %d, %Y')

In [20]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
ipos_filings['Shares Offered'] = pd.to_numeric(ipos_filings['Shares Offered'], errors='coerce')

In [21]:
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     324 non-null    object        
 4   Shares Offered  250 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [22]:
# Define a new field 'Avg_price' based on the "Price Range"

import re

def extract_prices(input_string):
  price_range = re.findall(r"[0.-9.]{1,7}", input_string)

  if len(price_range) == 1:
    return float(price_range[0])
  if len(price_range) == 2:
    return (float(price_range[0]) + float(price_range[1])) / 2
  else:
    return float('nan')

ipos_filings['Avg_price'] = ipos_filings['Price Range'].apply(lambda x:extract_prices(x))

In [23]:
# Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)


ipos_filings['Shares_offered_value'] = np.where(ipos_filings['Avg_price'] == float('nan'), float('nan'), ipos_filings['Shares Offered'] * ipos_filings['Avg_price'])


ipos_filings.head(13)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.00,NaN
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,5.00,NaN
7,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN,NaN,NaN
8,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN,NaN,NaN
9,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0,4.50,9000000.0


In [24]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
# You should see 32 records in total, 25 of it is not null.

ipos_filings['Shares_offered_value'][(ipos_filings['Filing Date'].dt.year == 2023) & (ipos_filings['Filing Date'].dt.dayofweek == 4)].sum() / 1000000

285.7

# End of Question 1

# Question 2

 Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB').

In [25]:
stackes_ipos_before_March1_df = stacked_ipos_df[(stacked_ipos_df['IPO Date'] < '2024-03-01') & (stacked_ipos_df['Symbol'] != 'RYZB')]

stackes_ipos_before_March1_df['Symbol'].replace(to_replace = 'PTHR', value = 'HOVR', inplace = True)

<ipython-input-25-509303b9f9c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stackes_ipos_before_March1_df['Symbol'].replace(to_replace = 'PTHR', value = 'HOVR', inplace = True)


In [26]:
symbols_list = stackes_ipos_before_March1_df['Symbol'].tolist()

growth_future_df = pd.DataFrame(columns=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30'])

for symbol in symbols_list:
  ipos_before_march_1 = yf.download(tickers = symbol,
                                  start = "2023-01-01",
                                  end = "2024-05-01",
                                  interval = "1d")

  growth_future = []

  if len(ipos_before_march_1) >= 30:
    for i in range(1, 31):
      growth_future.append(ipos_before_march_1['Adj Close'][i] / ipos_before_march_1['Adj Close'][0])

    growth_future_df.loc[len(growth_future_df)] = growth_future

growth_future_df


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,0.732500,0.611250,0.610000,0.582500,0.657500,0.897500,0.911250,0.885000,1.087500,0.957500,...,0.987500,1.143750,1.370000,1.342500,1.648750,1.441250,1.412500,1.420000,1.545000,1.485000
1,1.051988,1.021407,0.844037,0.905199,0.868502,0.697248,0.666667,0.611621,0.672783,0.648318,...,0.538838,0.500917,0.488073,0.428135,0.366972,0.305199,0.274006,0.242813,0.229358,0.243425
2,1.000000,0.995833,1.000000,0.902083,0.875000,0.833333,0.781250,0.785417,0.760417,0.681250,...,0.633333,0.595833,0.579167,0.512500,0.520833,0.508333,0.564583,0.545833,0.497917,0.500000
3,1.155116,0.986799,0.830033,0.818482,0.803630,1.095710,0.955446,1.084158,0.976898,0.937294,...,0.795380,0.798680,0.646865,0.577558,0.590759,0.551485,0.594059,0.669967,0.775578,0.577558
4,1.026247,1.081365,1.118110,1.072441,1.060892,1.088714,1.054593,1.091339,1.095538,1.086089,...,1.111811,1.155906,1.142257,1.139633,1.150656,1.138058,1.165354,1.196850,1.235696,1.249869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1.003195,1.006390,1.000799,1.006390,0.998403,0.998403,0.987220,0.996805,0.982428,0.965655,...,0.975240,0.994409,0.974441,0.998403,0.972843,0.995367,0.995367,0.974441,1.015974,0.979233
178,0.951456,0.776699,0.703884,0.689320,0.635922,0.623786,0.674757,0.686651,0.623786,0.582524,...,0.546117,0.539806,0.544903,0.511165,0.500000,0.483010,0.449029,0.451456,0.439320,0.478155
179,1.019895,0.996335,0.978534,0.970157,0.970157,0.968063,0.962304,0.991099,0.978534,0.970681,...,1.081152,1.080628,1.051309,1.029843,0.996335,0.997906,0.988482,1.000524,1.010471,1.021990
180,0.998032,0.999016,0.998032,1.000984,1.002953,1.003937,1.003937,1.003937,1.002953,1.000984,...,1.008858,1.007874,1.006890,1.009843,1.009843,1.006890,1.006890,1.009843,1.009843,1.009843


In [28]:
# Descriptive Analytics of a dataset
growth_future_df.describe().loc['75%']

1     1.015726
2     1.020828
3     1.010923
4     1.010111
5     1.009926
6     1.007620
7     1.006888
8     1.008586
9     1.011779
10    1.011603
11    1.013850
12    1.017138
13    1.020168
14    1.019902
15    1.019827
16    1.018612
17    1.012274
18    1.015602
19    1.019658
20    1.015365
21    1.014457
22    1.037117
23    1.028688
24    1.038096
25    1.029643
26    1.037616
27    1.044495
28    1.044178
29    1.029179
30    1.024949
Name: 75%, dtype: float64